In [1]:
import os
import cv2
import numpy as np
from PIL import Image
import mediapipe as mp
import pandas as pd

from libreface.utils import get_frames_from_video_opencv
from libreface.AU_Recognition.inference import get_au_intensities_and_detect_aus
from libreface.Facial_Expression_Recognition.inference import get_facial_expression


In [2]:
video_path = "D:/ATHARV/W/CDAC/CODE/LIBREFACE_IMPLEMENT/Data/sample_video.avi"
temp_dir = "./tmp"
device = "cpu"
output_dir = "D:/ATHARV/W/CDAC/CODE/LIBREFACE_IMPLEMENT/Notebooks/results_landmarks"
weights_dir = "./weights_libreface"
os.makedirs(output_dir, exist_ok=True)

In [3]:
def get_aligned_image(image_path, temp_dir="./tmp"):
    os.makedirs(temp_dir, exist_ok=True)
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Cannot read image: {image_path}")
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    mp_face_mesh = mp.solutions.face_mesh
    with mp_face_mesh.FaceMesh(static_image_mode=True,
                                refine_landmarks=True,
                                max_num_faces=1,
                                min_detection_confidence=0.5) as face_mesh:
        results = face_mesh.process(image_rgb)
        if not results.multi_face_landmarks:
            raise ValueError("No face landmarks detected")

        face_landmarks = results.multi_face_landmarks[0]

    img_h, img_w, _ = image.shape
    landmark_dict = {
        f"landmark_{i}": (lm.x, lm.y)
        for i, lm in enumerate(face_landmarks.landmark)
    }

    face_2d = []
    face_3d = []
    for idx, lm in enumerate(face_landmarks.landmark):
        if idx in [33, 263, 1, 61, 291, 199]:  # Nose, eyes, mouth corners
            x, y = int(lm.x * img_w), int(lm.y * img_h)
            face_2d.append([x, y])
            face_3d.append([x, y, lm.z])

    face_2d = np.array(face_2d, dtype=np.float64)
    face_3d = np.array(face_3d, dtype=np.float64)
    focal_length = 1 * img_w
    cam_matrix = np.array([
        [focal_length, 0, img_h / 2],
        [0, focal_length, img_w / 2],
        [0, 0, 1]
    ])
    dist_matrix = np.zeros((4, 1), dtype=np.float64)
    success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)
    rmat, _ = cv2.Rodrigues(rot_vec)
    angles, _, _, _, _, _ = cv2.RQDecomp3x3(rmat)
    pitch, yaw, roll = angles[0] * 360, angles[1] * 360, angles[2] * 360
    head_pose = {"pitch": pitch, "yaw": yaw, "roll": roll}

    pil_image = Image.fromarray(image_rgb)
    resized_image = pil_image.resize((256, 256), Image.Resampling.LANCZOS)
    aligned_name = os.path.splitext(os.path.basename(image_path))[0] + "_aligned.png"
    aligned_path = os.path.join(temp_dir, aligned_name)
    resized_image.save(aligned_path)

    return aligned_path, head_pose, landmark_dict

def draw_aus_on_frame(frame, aus: dict):
    y0, dy = 30, 30
    for i, (au, intensity) in enumerate(aus.items()):
        y = y0 + i * dy
        label = f"{au}: {intensity}"
        cv2.putText(frame, label, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 
                    1.2 , (0, 255, 0), 2, cv2.LINE_AA)
    return frame

def draw_landmarks(frame, landmark_dict):
    for name, (x, y) in landmark_dict.items():
        cx, cy = int(x * frame.shape[1]), int(y * frame.shape[0])
        cv2.circle(frame, (cx, cy), 2, (0, 0, 255), -1)
    return frame

def draw_expression(frame, expression):
    label = f"Expression: {expression}"
    cv2.putText(frame, label, (10, frame.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX,
                1.1, (255, 0, 0), 2, cv2.LINE_AA)
    return frame

In [8]:
print("Extracting frames...")
frames_df = get_frames_from_video_opencv(video_path, temp_dir=temp_dir)
frames_df = frames_df.iloc[::2]

if frames_df.empty:
    print("No frames extracted.")
    exit()

processed_frames = []
results = []

print("Processing frames...")
for _, row in frames_df.iterrows():
    frame_path = row['path_to_frame']
    frame = cv2.imread(frame_path)
    if frame is None:
        print(f"Could not read: {frame_path}")
        continue

    try:
        aligned_path, head_pose, landmark_dict = get_aligned_image(frame_path)

        detected_aus, au_intensities = get_au_intensities_and_detect_aus(
            image_path=frame_path,
            device=device,
            weights_download_dir=weights_dir
        )

        expression = get_facial_expression(
            aligned_path,
            device=device,
            weights_download_dir=weights_dir
        )

        annotated_data = {au: f"{au_intensities[au]:.2f}" for au in au_intensities}
        annotated_frame = draw_aus_on_frame(frame.copy(), annotated_data)
        annotated_frame = draw_landmarks(annotated_frame, landmark_dict)
        annotated_frame = draw_expression(annotated_frame, expression)

        pose_label = f"Pitch: {head_pose['pitch']:.1f}  Yaw: {head_pose['yaw']:.1f}  Roll: {head_pose['roll']:.1f}"
        cv2.putText(annotated_frame, pose_label, (10, frame.shape[0] - 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2, cv2.LINE_AA)

        # Store annotated frame
        processed_frames.append(annotated_frame)

        results.append({                
            "frame": os.path.basename(frame_path),
            "expression": expression,
            "pitch": round(head_pose["pitch"], 2),
            "yaw": round(head_pose["yaw"], 2),
            "roll": round(head_pose["roll"], 2),
            **{f"{au}_detected": int(detected_aus[au]) for au in detected_aus},
            **{f"{au}_intensity": float(f"{au_intensities[au]:.2f}") for au in au_intensities}
        })

    except Exception as e:
        print(f"Failed: {frame_path} – {e}")

# --- Display All Annotated Frames Like a Video ---
print("Playing back annotated video...")
cv2.namedWindow("Annotated Video", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Annotated Video", 1024, 768)

for frame in processed_frames:
    cv2.imshow("Annotated Video", frame)
    key = cv2.waitKey(30) & 0xFF  # ~30 FPS
    if key == ord('q') or key == 27:
        break

cv2.destroyAllWindows()

# --- Save results summary ---
csv_path = os.path.join(output_dir, "alt_results_summary.csv")  
pd.DataFrame(results).to_csv(csv_path, index=False)
print(f"\nCSV saved to: {csv_path}")
print("AU detection complete.")

Extracting frames...


Processing frames...
Playing back annotated video...

CSV saved to: D:/ATHARV/W/CDAC/CODE/LIBREFACE_IMPLEMENT/Notebooks/results_landmarks\alt_results_summary.csv
AU detection complete.
